**Retrieval augumented generation**
  * 1. Reterival relevant documents from a knowledge base.
  * 2. Generate an answer using a language model (like GPT) using those documents as context.

**Vector Store**
   * It's like a smart search engine.
   * Each chunk of your document is converted into a vector using an embedding model.
   * The vectors are stored in special databases (like FAISS) that can quickly find similar vectors for any question.

**Rerieval**
  * The retrieval searches the vector store for the most relavant chuncks of information (based on similarity).
  

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.1 MB/s eta 0:00:00


In [ ]:
import os
import fitz
from openai import OpenAI

In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

* OpenAIEmbeddings - convert text into vectors
* FAISS - A fast search engine for vectors
* Textsplitter - Break long text text into smaller chunks
* RetrievalQA - Combines retrieval and GPT answering
* ChatOpenAI - Allows calling GPT models like (GPT-4).

In [ ]:
#loading our PDF
def load_pdf_text(pdf_path):
  doc = fitz.open(pdf_path)
  full_text = ""
  for page in doc:
    text = page.get_text()
    full_text += text
  return full_text

In [ ]:
#Split_text
def split_text(text):
  splitter = RecursiveCharacterTextSplitter(chunk_size=300 , chunk_overlap=100)
  docs = splitter.create_documents([text])
  return docs

In [ ]:
#Create vector store
#each chunk of text converted into vector using OpenAI embeddings
#FAISS stores those vectors and allows fast similarity search
def create_vector_store(docs):
  embeddings = OpenAIEmbeddings()
  vectorstore = FAISS.from_documents(docs, embeddings)
  return vectorstore

In [ ]:
#Setup RAG pipeline
#retriever - pulls top 3 most similar chunks to question
#llm - GPT model that will generate an answer using those chunks
#RetrievalQA - combines them into a Question answering system

def setup_rag_qa(vectorstore):
  retriever = vectorstore.as_retriever(search_type="similarity" , search_kwargs={"k":3})
  llm = ChatOpenAI(temperature=0.3, model="gpt-4.1-nano")
  rag_chain = RetrievalQA.from_chain_type(llm=llm,
                                          retriever=retriever)
  return rag_chain

In [ ]:
!pip install faiss-cpu

In [ ]:
#Final execution
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('o_key')
pdf_path = '/content/company_manual.pdf'
text = load_pdf_text(pdf_path)
docs = split_text(text)
vectorstore = create_vector_store(docs)
qa_chain = setup_rag_qa(vectorstore)

#Ask a question
query = "Tell me about the company and product. Output expected in bullet form."
result = qa_chain(query)

print("Response:")
print(result['result'])

Response:
- TechNova Solutions Pvt. Ltd. is a global leader in consumer electronics.  
- The company is committed to delivering innovative technology and excellent customer service.  
- Headquarters are located in Bengaluru, India, with major regional offices in Singapore, Germany, and the United States.  
- The company specializes in smart devices, home automation, and sustainable technology solutions.  
- Its mission is to make everyday life smarter, simpler, and more connected.  
- TechNova values innovation, environmental responsibility, and customer satisfaction above all else.


In [ ]:
#Ask a question
query = "Tell me about the contact details of customer support. Output expected in bullet form."
result = qa_chain(query)

print("Response:")
print(result['result'])

Response:
- Phone: +91-9999999999 (Available Mon-Fri, 9 AM to 6 PM IST)
- Email: support@technova.com (Response time: within 24 hours)
- Live Chat: Available on our website and mobile app
- Help Center: https://support.technova.com


In [ ]:
'''
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Rag Assistant with GPT")
    gr.Markdown("Upload a PDF, then ask your question. Powered Bi OpenAI + FAISS + langchain")
    with gr.row():
      pdf_input = gr.file(label = "Upload PDF")
      upload_staus = gr.Textbox(label="Status")

    with gr.row():
      question_input = gr.Textbox(label = "Ask a question")
      answer_output = gr.Textbox(label = "Answer")

pdf_input.change(fn = , inputs = pdf_input,
                 output = upload_staus,
                 question_input.submit(fn = , input = question_input, output = answer_output))

demo.launch()
'''